In [1]:
# Задание на повторение материала предыдущего семестра

In [2]:
# Зависимости
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.cluster import KMeans

from sklearn.metrics import mean_squared_error, f1_score, silhouette_score

In [3]:
# Генерируем уникальный seed
my_code = "Johnson"
seed_limit = 2 ** 32
my_seed = int.from_bytes(my_code.encode(), "little") % seed_limit

In [4]:
# Данные загружены отсюда: https://www.kaggle.com/dwdkills/russian-demography
# Читаем данные из файла
example_data = pd.read_csv("datasets/russian_demography.csv")

In [5]:
# "year" - год (1990-2017)
# "region" - название региона
# "npg" - естественный прирост населения на 1000 человек
# "birth_rate" - количество рождений на 1000 человек
# "death_rate" - количество смертей на 1000 человек
# "gdw" - коэффициент демографической нагрузки на 100 человек (Отношение числа нетрудоспособных к числу трудоспособных).
# "urbanization" - процент городского населения

example_data.head()

,year,region,npg,birth_rate,death_rate,gdw,urbanization
0,1990,Republic of Adygea,1.9,14.2,12.3,84.66,52.42
1,1990,Altai Krai,1.8,12.9,11.1,80.24,58.07
2,1990,Amur Oblast,7.6,16.2,8.6,69.55,68.37
3,1990,Arkhangelsk Oblast,3.7,13.5,9.8,73.26,73.63
4,1990,Astrakhan Oblast,4.7,15.1,10.4,77.05,68.01


In [6]:
# Так как список регионов меняется от года к году, в данных есть строки без значений. Удалим их
example_data.dropna(inplace=True)

In [7]:
# Определим размер валидационной и тестовой выборок
val_test_size = round(0.2*len(example_data))
print(val_test_size)

463


In [8]:
# Создадим обучающую, валидационную и тестовую выборки
random_state = my_seed
train_val, test = train_test_split(example_data, test_size=val_test_size, random_state=random_state)
train, val = train_test_split(train_val, test_size=val_test_size, random_state=random_state)
print(len(train), len(val), len(test))

1389 463 463


In [9]:
# Значения в числовых столбцах преобразуем к отрезку [0,1].
# Для настройки скалировщика используем только обучающую выборку.
columns_to_scale = ['year', 'npg', 'birth_rate', 'death_rate', 'gdw', 'urbanization']

ct = ColumnTransformer(transformers=[('numerical', MinMaxScaler(), columns_to_scale)], remainder='passthrough')
ct.fit(train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('numerical', MinMaxScaler(),
                                 ['year', 'npg', 'birth_rate', 'death_rate',
                                  'gdw', 'urbanization'])])

In [10]:
# Преобразуем значения, тип данных приводим к DataFrame
sc_train = pd.DataFrame(ct.transform(train))
sc_test = pd.DataFrame(ct.transform(test))
sc_val = pd.DataFrame(ct.transform(val))

In [11]:
# Устанавливаем названия столбцов
column_names = columns_to_scale + ['region']
sc_train.columns = column_names
sc_test.columns = column_names
sc_val.columns = column_names

In [12]:
sc_train

,year,npg,birth_rate,death_rate,gdw,urbanization,region
0,0.518519,0.313625,0.25,0.562212,0.352911,0.605866,Udmurt Republic
1,0.777778,0.429306,0.313725,0.414747,0.347176,0.611104,Tomsk Oblast
2,0.888889,0.269923,0.22549,0.617512,0.645352,0.708001,Vladimir Oblast
3,0.925926,0.393316,0.328431,0.493088,0.569765,0.571821,Amur Oblast
4,0.925926,0.624679,0.539216,0.276498,0.526325,0.629436,Nenets Autonomous Okrug
...,...,...,...,...,...,...,...
1384,0.481481,0.0719794,0.0686275,0.820276,0.48497,0.560429,Leningrad Oblast
1385,0.481481,0.547558,0.681373,0.552995,0.550478,0.367946,Tuva Republic
1386,0.111111,0.257069,0.112745,0.534562,0.731017,0.521409,Penza Oblast
1387,0.148148,0.241645,0.137255,0.585253,0.610599,0.651172,Arkhangelsk Oblast


In [13]:
# Вспоминаем алгоритмы решения задачи регрессии: линейную регрессию и метод k ближайших соседей
r_models = []

# Линейная регрессия
# Для использования регуляризации, вместо LinearRegression используем Lasso, Ridge или ElasticNet
# Параметр alpha - коэффициент регуляризации для Lasso и Ridge, по умолчанию равен 1
# Для ElasticNet, если регуляризация иммет вид a*L1+b*L2, то
# параметр alpha = a + b, по умолчанию равен 1
# параметр l1_ratio = a / (a + b), по умолчанию равен 0.5
r_models.append(LinearRegression())
r_models.append(Lasso(alpha=1.0))
r_models.append(Ridge(alpha=1.0))
r_models.append(ElasticNet(alpha=1.0, l1_ratio=0.5))

# K ближайших соседей
# Параметр n_neighbors - количество соседей, по умолчания равен 5
r_models.append(KNeighborsRegressor(n_neighbors=5))
r_models.append(KNeighborsRegressor(n_neighbors=10))
r_models.append(KNeighborsRegressor(n_neighbors=15))

In [14]:
# Выделим предикторы и зависимую переменную
x_labels = column_names[0:-2]
y_labels = ['urbanization']

x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = sc_train[y_labels]
y_test = sc_test[y_labels]
y_val = sc_val[y_labels]

In [15]:
# Обучаем модели
for model in r_models:
    model.fit(x_train, y_train)

In [16]:
# Оценииваем качество работы моделей на валидационной выборке.
mses = []
for model in r_models:
    val_pred = model.predict(x_val)
    mse = mean_squared_error(y_val, val_pred)
    mses.append(mse)
    print(mse)

0.01758075517904994
0.02599402772650842
0.017566576927921986
0.02599402772650842
0.009713831257495904
0.010929161584457113
0.011900501932615905


In [17]:
# Выбираем лучшую модель
i_min = mses.index(min(mses))
best_r_model = r_models[i_min]
best_r_model

KNeighborsRegressor()

In [18]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_r_model.predict(x_test)
mse = mean_squared_error(y_test, test_pred)
print(mse)

0.010757270417534362


In [19]:
# Вспоминаем алгоритмы решения задачи классификации:
# логистическую регрессию, наивный байесовский классификатор и (снова) метод k ближайших соседей
c_models = []

# Логистическая регрессия
# Параметр penalty - тип регуляризации: 'l1', 'l2', 'elasticnet', 'none'}, по умолчанию 'l2'
# Для некоторых типов регуляризации доступны не все алгоритмы (параметр solver)
# Для elasticnet регуляризации необходимо уазывать параметр l1_ratio (0 - l2, 1 - l1)
c_models.append(LogisticRegression(penalty='none', solver='saga'))
c_models.append(LogisticRegression(penalty='l1', solver='saga'))
c_models.append(LogisticRegression(penalty='l2', solver='saga'))
c_models.append(LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga'))
c_models.append(LogisticRegression())

# Наивный байесовский классификатор
# Параметр alpha - параметр сглаживания, по умолчанию равен 1 (сглаживание Лапласа)
c_models.append(MultinomialNB(alpha=0.0))
c_models.append(MultinomialNB(alpha=0.5))
c_models.append(MultinomialNB(alpha=1.0))

# K ближайших соседей
# Параметр n_neighbors - количество соседей, по умолчания равен 5
c_models.append(KNeighborsClassifier(n_neighbors=5))
c_models.append(KNeighborsClassifier(n_neighbors=10))
c_models.append(KNeighborsClassifier(n_neighbors=15))

In [20]:
# Выделим предикторы и метки классов
x_labels = column_names[0:-1]
y_labels = ['region']

x_train = sc_train[x_labels]
x_test = sc_test[x_labels]
x_val = sc_val[x_labels]

y_train = np.ravel(sc_train[y_labels])
y_test = np.ravel(sc_test[y_labels])
y_val = np.ravel(sc_val[y_labels])

In [21]:
# Обучаем модели
for model in c_models:
    model.fit(x_train, y_train)

H:\Anaconda\envs\keras_data_processing\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
H:\Anaconda\envs\keras_data_processing\lib\site-packages\sklearn\linear_model\_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
H:\Anaconda\envs\keras_data_processing\lib\site-packages\sklearn\naive_bayes.py:512: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


In [22]:
# Оценииваем качество работы моделей на валидационной выборке.
f1s = []
for model in c_models:
    val_pred = model.predict(x_val)
    f1 = f1_score(y_val, val_pred, average='weighted')
    f1s.append(f1)
    print(f1)

0.2863190588514729
0.11690918555040014
0.05052985017664285
0.05172478680021772
0.05052727983887178
0.02599420462130576
0.02785474416459065
0.025936016831200594
0.33932622556669434
0.24560724429229064
0.2020326209040276


In [23]:
# Выбираем лучшую модель
i_min = f1s.index(min(f1s))
best_c_model = c_models[i_min]
best_c_model

MultinomialNB()

In [24]:
# Вычислим ошибку лучшей модели на тестовой выборке.
test_pred = best_c_model.predict(x_test)
f1 = f1_score(y_test, test_pred, average='weighted')
print(f1)

0.028343786328518392


In [25]:
# Вспоминаем алгоритм решения задачи кластеризации - метод k-средних
# Параметр n_clusters - количество кластеров, по умолчанию равен 8
k_models = []
k_models.append(KMeans(n_clusters=5))
k_models.append(KMeans(n_clusters=8))
k_models.append(KMeans(n_clusters=20))
k_models.append(KMeans(n_clusters=50))

In [26]:
# Выделим используемые параметры
x_labels = column_names[0:-1]
x = pd.concat([sc_train[x_labels], sc_val[x_labels], sc_test[x_labels]])
x

,year,npg,birth_rate,death_rate,gdw,urbanization
0,0.518519,0.313625,0.25,0.562212,0.352911,0.605866
1,0.777778,0.429306,0.313725,0.414747,0.347176,0.611104
2,0.888889,0.269923,0.22549,0.617512,0.645352,0.708001
3,0.925926,0.393316,0.328431,0.493088,0.569765,0.571821
4,0.925926,0.624679,0.539216,0.276498,0.526325,0.629436
...,...,...,...,...,...,...
458,0.222222,0.251928,0.0833333,0.516129,0.723719,0.454105
459,0.259259,0.257069,0.0588235,0.483871,0.660122,0.642137
460,0.037037,0.77892,0.848039,0.290323,0.805387,0.310986
461,0,0.491003,0.401961,0.387097,0.748045,0.398193


In [27]:
# Произведем кластеризацию
for model in k_models:
    model.fit(x)

In [28]:
# Оценим качество результата
sils = []
for model in k_models:
    cluster_labels = model.predict(x)
    s = silhouette_score(x, cluster_labels)
    sils.append(s)
    print(s)

0.29281647456006854
0.251625399674726
0.23843703994480214
0.23192426641263317


In [29]:
# Выбираем лучшую модель
i_min = sils.index(min(sils))
best_k_model = k_models[i_min]
print(best_k_model)
print(sils[i_min])

KMeans(n_clusters=50)
0.23192426641263317


In [30]:
# Задание №1 - анализ моделей для задачи регрессии
# Общий список моделей
r_models = [
    LinearRegression(),
    Lasso(alpha=1.0),
    Lasso(alpha=0.5),
    Ridge(alpha=1.0),
    Ridge(alpha=0.5),
    ElasticNet(alpha=1.0, l1_ratio=0.5),
    ElasticNet(alpha=1.0, l1_ratio=0.25),
    ElasticNet(alpha=1.0, l1_ratio=0.75),
    ElasticNet(alpha=0.5, l1_ratio=0.5),
    ElasticNet(alpha=0.5, l1_ratio=0.25),
    ElasticNet(alpha=0.5, l1_ratio=0.75),
    KNeighborsRegressor(n_neighbors=5),
    KNeighborsRegressor(n_neighbors=10),
    KNeighborsRegressor(n_neighbors=15),
    KNeighborsRegressor(n_neighbors=20),
    KNeighborsRegressor(n_neighbors=25)
]

In [31]:
# Выбор моделей для задания
n = 4
random.seed(my_seed)
my_models1 = random.sample(r_models, n)
print(my_models1)

[Ridge(), ElasticNet(alpha=0.5), Lasso(), Lasso(alpha=0.5)]


In [32]:
# Загрузим данные для задачи регрессии
data = pd.read_csv("datasets/weather.csv")
data

,water_level,precipitation,temperature,humidity,visibility,wind,weather,pressure,fire,wl_change,temp_change,pressure_change
0,"468,0","0,0","24,9","31,0","100,0","4,0","0,0","993,8","4,0","-3,0","-0,3","-0,4"
1,"140,0","0,0","27,9","22,0","20,0","1,0","0,0","1004,1","4,0","-2,0","0,6","-1,0"
2,"464,0","0,0","25,6","28,0","20,0","2,0","1,0","1001,2","4,0","-7,0","-1,8","-0,3"
3,"467,0","0,0","26,7","26,0","100,0","3,0","0,0","992,3","4,0","-1,0","1,2","-1,3"
4,"138,0","0,0","29,1","22,0","20,0","1,0","0,0","1001,3","4,0","-2,0","2,5","-1,3"
...,...,...,...,...,...,...,...,...,...,...,...,...
1434,"474,0","0,1","6,4","95,0","4,0","1,0","3,0","999,9","4,0","-1,0","5,9","1,5"
1435,"65,0","0,0","6,4","93,0","10,0","4,0","3,0","983,9","4,0","-1,0","5,2","3,1"
1436,"159,0","0,1","8,5","83,0","10,0","2,0","1,0","969,7","4,0","1,0","1,4","1,9"
1437,"99,0","0,1","7,5","91,0","10,0","3,0","3,0","995,4","4,0","-4,0","2,7","1,7"


In [33]:
# Зависимая переменная для всех одна и та же. Предикторы выбираем случайнм образом.
columns = list(data.columns)
n_x = 5

y_label = 'water_level'
x_labels = random.sample(columns[1:], n_x)

print(x_labels)

['temp_change', 'pressure', 'precipitation', 'humidity', 'fire']


In [34]:
# Преобразуйте значения всех необходимых параметров к отрезку [0,1].
# Решите получившуюся задачу регрессии с помощью выбранных моделей и сравните их эффективность.
# Укажите, какая модель решает задачу лучше других.

In [35]:
# Задание №2 - анализ моделей для задачи классификации
# Общий список моделей
c_models = [
    LogisticRegression(penalty='none', solver='saga'),
    LogisticRegression(penalty='l1', solver='saga'),
    LogisticRegression(penalty='l2', solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.25, solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga'),
    LogisticRegression(penalty='elasticnet', l1_ratio=0.75, solver='saga'),
    LogisticRegression(),
    MultinomialNB(alpha=0.0),
    MultinomialNB(alpha=0.25),
    MultinomialNB(alpha=0.5),
    MultinomialNB(alpha=0.75),
    MultinomialNB(alpha=1.0),
    KNeighborsClassifier(n_neighbors=5),
    KNeighborsClassifier(n_neighbors=10),
    KNeighborsClassifier(n_neighbors=15),
    KNeighborsClassifier(n_neighbors=20),
    KNeighborsClassifier(n_neighbors=25)
]

In [36]:
# Выбор моделей для задания
n = 5
my_models2 = random.sample(c_models, n)
print(my_models2)

[LogisticRegression(penalty='none', solver='saga'), KNeighborsClassifier(n_neighbors=15), KNeighborsClassifier(n_neighbors=20), LogisticRegression(l1_ratio=0.5, penalty='elasticnet', solver='saga'), LogisticRegression(l1_ratio=0.25, penalty='elasticnet', solver='saga')]


In [37]:
# Загрузим данные для задачи классификации
data = pd.read_csv("datasets/zoo2.csv")
data

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,turtle,0,0,1,0,0,1,0,0,1,1,0,0,4,1,1,1,3
1,chameleon,0,0,1,0,0,0,0,1,1,1,0,0,4,1,1,0,3
2,iguana,0,0,1,0,0,0,1,1,1,1,0,0,4,1,1,1,3
3,lizard,0,0,1,0,0,0,1,1,1,1,0,0,4,1,0,0,3
4,gecko,0,0,1,0,0,0,0,1,1,1,0,0,4,1,1,0,3
5,python,0,0,1,0,0,0,1,1,1,1,1,0,0,1,0,1,3
6,boa,0,0,1,0,0,0,1,1,1,1,0,0,0,1,0,1,3
7,adder,0,0,1,0,0,0,1,1,1,1,1,0,0,1,0,1,3
8,crocodile,0,0,1,0,0,1,1,1,1,1,0,0,4,1,0,1,3
9,alligator,0,0,1,0,0,1,1,1,1,1,0,0,4,1,0,1,3


In [38]:
# Метка класса для всех одна и та же. Параметры выбираем случайнм образом.
columns = list(data.columns)
n_x = 8

y_label = 'class_type'
x_labels = random.sample(columns[:-1], n_x)

print(x_labels)

['predator', 'hair', 'backbone', 'domestic', 'animal_name', 'fins', 'tail', 'airborne']


In [39]:
# Преобразуйте значения всех необходимых параметров к отрезку [0,1].
# Решите получившуюся задачу классификации с помощью выбранных моделей и сравните их эффективность.
# Укажите, какая модель решает задачу лучше других.